In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
N_JOBS = 15 # cpu_count()
os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
os.environ['OMP_NUM_THREADS'] = str(N_JOBS)

In [ ]:
# DO NOT DO THIS IN YOUR PROJETS
from config import *
from utils import *
from keras_utils import *
from augmentations import *

In [ ]:
model = UEfficientNet(input_shape=(IMG_SIZE,IMG_SIZE,3), dropout_rate=0.25)
model.load_weights(f'{CHECKPOINT_DIR}/keras.model_{TAG}')
model.compile(loss=bce_dice_loss, optimizer='adam', metrics=[my_iou_metric])

In [ ]:
valid_im_path,valid_mask_path = f'{DATA_DIR}/keras_im_val',f'{DATA_DIR}/keras_mask_val'


validation_generator = DataGenerator(train_im_path=valid_im_path,
                                     train_mask_path=valid_mask_path,
                                     augmentations=AUGMENTATIONS_TEST,
                                     img_size=IMG_SIZE,
                                     shuffle=False)
preds_valid = predict_result(model,
                             validation_generator,
                             IMG_SIZE)

In [ ]:
valid_fn = glob.glob(f'{DATA_DIR}/keras_mask_val/*')
y_valid_ori = np.array([cv2.resize(np.array(Image.open(fn)), (IMG_SIZE,IMG_SIZE)) for fn in valid_fn])
assert y_valid_ori.shape == preds_valid.shape

In [ ]:
## Scoring for best model
thresholds = np.linspace(0.4, 0.95, 16)
ious = np.array([iou_metric_batch(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in thresholds])

In [ ]:
threshold_best_index = np.argmax(ious) 
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
test_fn = glob.glob(f'{DATA_DIR}/test/*')
x_test = [cv2.resize(np.array(Image.open(fn)),(IMG_SIZE, IMG_SIZE)) for fn in test_fn]
x_test = np.array(x_test)
x_test = np.array([np.repeat(im[...,None],3,2) for im in x_test])
print(x_test.shape)
preds_test = model.predict(x_test,batch_size=batch_size)

In [ ]:
import pdb

rles = []
i,max_img = 1,10
for p in preds_test:
    p = p.squeeze()
    im = cv2.resize(p,(1024,1024))
    im = im > threshold_best
#     zero out the smaller regions.
    if im.sum()<1024*2:
        im[:] = 0
    im = (im.T*255).astype(np.uint8)  
    rles.append(mask2rle(im, 1024, 1024))


In [ ]:
ids = [o.split('/')[-1][:-4] for o in test_fn]
sub_df = pd.DataFrame({'ImageId': ids, 'EncodedPixels': rles})
sub_df.loc[sub_df.EncodedPixels=='', 'EncodedPixels'] = '-1'
sub_df.head()

In [ ]:
sub_df.to_csv(f'keras_submission_{TAG}_best.csv', index=False)

In [ ]:
sub_df.tail(10)

In [ ]:
quit()

In [ ]:
# for blending
# test_fn = glob.glob(f'{DATA_DIR}/test/*')
# x_test = [cv2.resize(np.array(Image.open(fn)),(IMG_SIZE, IMG_SIZE)) for fn in test_fn]
# x_test = np.array(x_test)
# x_test = np.array([np.repeat(im[...,None],3,2) for im in x_test])
# print(x_test.shape)
# preds_test = model.predict(x_test,batch_size=batch_size)
# # del x_test; gc.collect()
# # write_pickle_obj(preds_test, name='test_640.pickle')

# ids = [o.split('/')[-1][:-4] for o in test_fn]
# sub_df = pd.DataFrame({'ImageId': ids, 'EncodedPixels': -1})
# file_names = sub_df['ImageId']

# for i, (p) in enumerate(preds_test):
#     p = p.squeeze()
#     im = cv2.resize(p,(1024,1024))
#     cv2.imwrite(f'test_masks_sa/{file_names[i]}.png', im)